In [ ]:
%pip install requests openai

### 2. 檢查伺服器狀態並初始化客戶端

此儲存格執行以下操作：
1.  **定義伺服器端點**：設定 vLLM 伺服器的基本 URL。
2.  **檢查可用模型**：向 `/models` 端點發送 GET 請求，以擷取伺服器上可用的模型列表。
3.  **設定模型名稱**：從回應中提取模型名稱以供後續請求使用。如果伺服器無法連線或沒有模型可用，則會使用預設名稱。
4.  **初始化 OpenAI 客戶端**：建立 `OpenAI` 客戶端的實例，並將其設定為指向 vLLM 伺服器的 `base_url`。`api_key` 設定為虛擬金鑰，因為此本地伺服器不需要身份驗證。

In [ ]:
# 先檢查伺服器狀態
import requests

# 檢查可用模型
vllm_gpt_oss_120b_1="http://210.61.209.139:45014/v1/"
vllm_gpt_oss_120b_2="http://210.61.209.139:45005/v1/"

base_url = vllm_gpt_oss_120b_1
try:
    response = requests.get(base_url+"models")
    models = response.json()
    print("Available models:", models)
    
    # 取得實際的模型名稱
    if models.get("data"):
        model_name = models["data"][0]["id"]
        print(f"Using model: {model_name}")
    else:
        print("No models available!")
        model_name = "gpt-oss-120b"  # 預設值
except Exception as e:
    print(f"Error connecting to vLLM server: {e}")
    model_name = "gpt-oss-120b"

# 初始化 OpenAI 客戶端
from openai import OpenAI

client = OpenAI(
    base_url=base_url,
    api_key="dummy-key"
)

### 3. 文字生成測試

此儲存格使用 `client.completions.create` 方法測試基本文字生成功能。

它會：
1.  發送一個簡單的提示 (`"Once upon a time in a magical forest,"`)。
2.  列印生成的文字。

In [ ]:
# 文字生成測試 

message="Once upon a time in a magical forest,"

try:
    
    response = client.completions.create(
        model=model_name,
        prompt=message,
        max_tokens=100,
        temperature=0.8
    )
    

    
    generated_text = response.choices[0].text
    print("Prompt:", message)
    print("Generated text:", generated_text)
    
        
except Exception as e:
    print(f"Error: {e}")

### 4. 聊天補全測試

此儲存格使用 `client.chat.completions.create` 方法測試聊天功能。此方法更適合對話式或指令式的互動。

它會：
1.  發送包含系統訊息（設定助理的行為）和使用者訊息（問題）的對話歷史記錄。
2.  列印模型的聊天回應。

In [ ]:
# Chat completion 測試

try:
   
    chat_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Explain machine learning in one paragraph."}
        ],
        temperature=0.7
    )
    
    response_content = chat_response.choices[0].message.content
    print("Chat response:", response_content)
    
        
except Exception as e:
    print(f"Chat error: {e}")

### 5. 推理能力測試 - 複雜邏輯謎題

此儲存格旨在透過給予一個需要逐步推理的複雜邏輯謎題來評估模型的推理能力。

它會：
1.  使用系統提示，指示模型「逐步思考」。
2.  提供一個包含多個線索的邏輯謎題。
3.  將 `temperature` 設定為較低的值 (`0.1`)，以鼓勵更具確定性和邏輯性的一致回應。

In [ ]:
# Reasoning Effort 測試 - 複雜推理任務

try:
    
    reasoning_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert problem solver. Think step by step and show your reasoning process."},
            {"role": "user", "content": """
            Solve this logic puzzle step by step:
            
            Three friends - Alice, Bob, and Carol - each have a different pet (cat, dog, bird) and live in different colored houses (red, blue, green).
            
            Clues:
            1. Alice doesn't live in the red house
            2. The person with the cat lives in the blue house
            3. Bob doesn't have a bird
            4. Carol doesn't live in the green house
            5. The person in the red house has a dog
            
            Who has which pet and lives in which house?
            """}
        ],
        temperature=0.1,  # 低溫度確保邏輯一致性
        max_tokens=500
    )
    
    
    reasoning_content = reasoning_response.choices[0].message.content
    print("Reasoning Response:", reasoning_content)
    

except Exception as e:
    print(f"Reasoning error: {e}")

### 6. 推理能力測試 - 數學問題

此儲存格透過一個需要基本數學計算的簡單問題，繼續測試模型的推理能力。

它會：
1.  要求模型解決一個關於速度和距離的數學應用題，並「展示其解題過程」。
2.  此測試中的 `reasoning_effort="high"` 是一個範例參數，用於展示如何向 API 發出需要更強推理能力的信號（請注意，此參數可能並非在所有 vLLM 設定中都為標準功能）。

In [ ]:
# 簡化推理測試 - 數學問題

try:

    system_prompt="You are a helpful math tutor. Solve problems step by step."
    user_prompt="A train travels 120 km in 2 hours. If it maintains the same speed, how far will it travel in 5 hours? Please show your work."
    
    math_response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=200,
        reasoning_effort="high"
    )

    
    math_content = math_response.choices[0].message.content
    print("\n")
    print("Prompt:", user_prompt)
    print("\nMath Reasoning Response:", math_content)
    
        
except Exception as e:
    print(f"Math reasoning error: {e}")

### 7. 模型診斷測試

此最後一個儲存格執行快速診斷，以檢查模型輸出的基本品質和健全性。

它會：
1.  迭代一組簡單、多樣的提示（一個數學問題、一個解釋性問題和一個列表請求）。
2.  對於每個提示，它會擷取回應並測量延遲。
3.  執行基本的品質檢查：
    - 確保回應不為空。
    - 檢查字元多樣性，以標記潛在的重複性問題（例如，模型輸出重複的字元）。
4.  根據檢查結果列印成功或警告訊息。

In [ ]:
# 模型診斷測試 - 檢查輸出品質

try:
    # 測試不同的提示策略
    prompts = [
        "What is 2+2?",
        "Explain photosynthesis briefly.",
        "List 3 colors."
    ]
    
    for i, prompt in enumerate(prompts, 1):
        print(f"\n--- Test {i}: {prompt} ---")

        
        response = client.chat.completions.create(
            model=model_name,
            messages=[ 
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=50
        )
        
 
        
        content = response.choices[0].message.content
        print(f"Response: {content}")
        
        # 檢查輸出品質
        if content and len(content.strip()) > 0:
            # 檢查是否有重複字符（如感嘆號問題）
            if len(set(content)) < 5:  # 如果字符種類太少
                print("⚠️ Warning: Low character diversity - possible repetition issue")
            else:
                print("✅ Output looks normal")
        else:
            print("❌ Empty or invalid response")
            
except Exception as e:
    print(f"Diagnostic error: {e}")